In [1]:
import pandas as pd
import os
from sklearn.metrics import classification_report
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

In [11]:
import sys
sys.path.append('../src')

from models.classification_methods import get_classification_report

In [2]:
test_results_path = '../reports/test_results/'

list_df_t = os.listdir(test_results_path)
list_df_t.sort()
list_df_t

['belt_filteredTimeline10_pablocosta_bertabaporu-base-uncased_test_results.csv',
 'belt_filteredTimeline10_pablocosta_bertabaporu-base-uncased_test_results_part_bo.csv',
 'belt_filteredTimeline10_pablocosta_bertabaporu-base-uncased_test_results_part_cl.csv',
 'belt_filteredTimeline10_pablocosta_bertabaporu-base-uncased_test_results_part_co.csv',
 'belt_filteredTimeline10_pablocosta_bertabaporu-base-uncased_test_results_part_gl.csv',
 'belt_filteredTimeline10_pablocosta_bertabaporu-base-uncased_test_results_part_ig.csv',
 'belt_filteredTimeline10_pablocosta_bertabaporu-base-uncased_test_results_part_lu.csv',
 'belt_filteredTimeline15_pablocosta_bertabaporu-base-uncased_test_results.csv',
 'belt_filteredTimeline15_pablocosta_bertabaporu-base-uncased_test_results_part_bo.csv',
 'belt_filteredTimeline15_pablocosta_bertabaporu-base-uncased_test_results_part_cl.csv',
 'belt_filteredTimeline15_pablocosta_bertabaporu-base-uncased_test_results_part_co.csv',
 'belt_filteredTimeline15_pablocosta_

In [31]:
# Target list
target_list = [
    'ig',
    'bo', 
    'cl', 
    'co', 
    'gl', 
    'lu'
]

dict_cp = {
    'cl':'Hidroxicloroquina',
    'lu':'Lula',
    'co':'Sinovac',
    'ig':'Church',
    'gl':'Globo TV',
    'bo':'Bolsonaro',
}

In [37]:
# (vectorizer,estimator, path_sring) 
results_tuples_stance = [
    ("-" ,"bertabaporu-base-uncased", "bert_classifier_pablocosta_bertabaporu_base_uncased_{target}_Stance_test_results.csv")
]

for vectorizer, estimator, path_results in results_tuples_stance:
    
    list_cr = []
    
    for target in target_list:
        
        df_results = pd.read_csv(test_results_path + path_results.format(target = target))
        
        
        # get classification report df
        df_classification_report = get_classification_report(df_results.test, df_results.pred, cr_args = {})
        
        # create multindex
        column_indexes = [(metric,dict_cp[target]) for metric in df_classification_report.columns]
        multi_index_cols = pd.MultiIndex.from_tuples(column_indexes, names=['metric', 'target'])
        rows_indexes = [(vectorizer, estimator, cl) for cl in df_classification_report.index]
        multi_index_rows = pd.MultiIndex.from_tuples(rows_indexes, names=['vectorizer', 'estimator', 'class'])
        df_classification_report.columns = multi_index_cols
        df_classification_report.index = multi_index_rows
        
        
        
        list_cr.append(df_classification_report)
        
    df_results_final = pd.concat(list_cr, axis = 1)

In [38]:
df_results_final

metric                                           precision    recall  \
target                                              Church    Church   
vectorizer estimator                class                              
-          bertabaporu-base-uncased against       0.850267  0.938053   
                                    accuracy      0.871452  0.871452   
                                    weighted avg  0.874748  0.871452   
                                    macro avg     0.878467  0.861334   
                                    for           0.906667  0.784615   

metric                                            f1-score     support  \
target                                              Church      Church   
vectorizer estimator                class                                
-          bertabaporu-base-uncased against       0.892006  339.000000   
                                    accuracy      0.871452    0.871452   
                                    weighted avg  0.869969  599.000000   
                                    macro avg     0.866621  599.000000   
                                    for           0.841237  260.000000   

metric                                           precision    recall  \
target                                           Bolsonaro Bolsonaro   
vectorizer estimator                class                              
-          bertabaporu-base-uncased against       0.926380  0.932099   
                                    accuracy      0.877660  0.877660   
                                    weighted avg  0.875711  0.877660   
                                    macro avg     0.743190  0.735280   
                                    for           0.560000  0.538462   

metric                                            f1-score    support  \
target                                           Bolsonaro  Bolsonaro   
vectorizer estimator                class                               
-          bertabaporu-base-uncased against       0.929231  162.00000   
                                    accuracy      0.877660    0.87766   
                                    weighted avg  0.876648  188.00000   
                                    macro avg     0.739125  188.00000   
                                    for           0.549020   26.00000   

metric                                                   precision  \
target                                           Hidroxicloroquina   
vectorizer estimator                class                            
-          bertabaporu-base-uncased against               0.875940   
                                    accuracy              0.844948   
                                    weighted avg          0.847262   
                                    macro avg             0.847061   
                                    for                   0.818182   

metric                                                      recall  ...  \
target                                           Hidroxicloroquina  ...   
vectorizer estimator                class                           ...   
-          bertabaporu-base-uncased against               0.806228  ...   
                                    accuracy              0.844948  ...   
                                    weighted avg          0.844948  ...   
                                    macro avg             0.845219  ...   
                                    for                   0.884211  ...   

metric                                            f1-score     support  \
target                                             Sinovac     Sinovac   
vectorizer estimator                class                                
-          bertabaporu-base-uncased against       0.823188  354.000000   
                                    accuracy      0.842377    0.842377   
                                    weighted avg  0.841975  774.000000   
                                    macro avg     0.840499  77

In [50]:
mask_f1 = [True if  "f1-score" in col else False for col in df_results_final.columns]
mask_macro = [True if  "macro avg" in col else False for col in df_results_final.index]

In [56]:
f1_macro_df = df_results_final.loc[mask_macro,mask_f1]
f1_macro_df

metric                                         f1-score            \
target                                           Church Bolsonaro   
vectorizer estimator                class                           
-          bertabaporu-base-uncased macro avg  0.866621  0.739125   

metric                                                                     \
target                                        Hidroxicloroquina   Sinovac   
vectorizer estimator                class                                   
-          bertabaporu-base-uncased macro avg          0.844778  0.840499   

metric                                                             
target                                         Globo TV      Lula  
vectorizer estimator                class                          
-          bertabaporu-base-uncased macro avg  0.862638  0.796338